In [ ]:
!pip install kaggle


In [ ]:
import os
import shutil

# Make directory for Kaggle configuration
os.makedirs('/root/.kaggle/', exist_ok=True)

# Move the kaggle.json file to the correct location
shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')

# Set permissions for the kaggle.json file
os.chmod('/root/.kaggle/kaggle.json', 600)


**Download Dataset to colab from kaggle**

In [ ]:
!kaggle datasets download -d netflix-inc/netflix-prize-data -p /content


Dataset URL: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data
License(s): other
 97% 665M/683M [00:06<00:00, 118MB/s]
100% 683M/683M [00:06<00:00, 115MB/s]


**Extract the data**

In [ ]:
import zipfile

zip_path = '/content/netflix-prize-data.zip'
extract_path = '/content/netflix-prize-data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
!pip install pymongo

**Connect to mongodb**

In [ ]:
from pymongo import MongoClient

# Replace with your MongoDB connection details
client = MongoClient('mongodb://192.168.0.112:8081/')
db = client['netflix']


**Store movie titles**

In [ ]:
import csv

movies_collection = db['movies']

batch_size = 10000  # Define your batch size
batch = []

with open('/content/netflix-prize-data/movie_titles.csv', 'r', encoding='ISO-8859-1') as file:
    reader = csv.reader(file, delimiter=',')
    for row in reader:
        movie_id = int(row[0])
        year = row[1]
        title = row[2]

        # Handle 'NULL' year
        year = int(year) if year != 'NULL' else None

        movie_doc = {
            "movie_id": movie_id,
            "year_of_release": year,
            "title": title
        }
        batch.append(movie_doc)

        if len(batch) >= batch_size:
            movies_collection.insert_many(batch)
            batch.clear()

    # Insert remaining documents in the batch
    if batch:
        movies_collection.insert_many(batch)


**store ratings**

In [ ]:
import os
from tqdm import tqdm

ratings_collection = db['ratings']

ratings_files = ['combined_data_1.txt', 'combined_data_2.txt', 'combined_data_3.txt', 'combined_data_4.txt']
batch_size = 100000  # Define your batch size
batch = []

for filename in tqdm(ratings_files):
    with open(os.path.join('/content/netflix-prize-data/', filename), 'r') as file:
        lines = file.readlines()
        current_movie_id = None
        for line in lines:
            if line.endswith(':\n'):  # Movie ID line
                current_movie_id = int(line.strip().replace(':', ''))
            else:
                try:
                    customer_id, rating, date = line.strip().split(',')
                    rating_doc = {
                        "movie_id": current_movie_id,
                        "customer_id": int(customer_id),
                        "rating": int(rating),
                        "date": date
                    }
                    batch.append(rating_doc)
                except ValueError as e:
                    print(f"Skipping line due to parsing error: {line} - Error: {e}")

                if len(batch) >= batch_size:
                    ratings_collection.insert_many(batch)
                    batch.clear()

        # Insert remaining documents in the batch
        if batch:
            ratings_collection.insert_many(batch)
            batch.clear()


100%|██████████| 4/4 [1:51:34<00:00, 1673.53s/it]


**store probe and qualifying data**

In [ ]:
import os
from tqdm import tqdm

qualifying_collection = db['qualifying']
probe_collection = db['probe']

# Define file paths
qualifying_file = 'qualifying.txt'
probe_file = 'probe.txt'
data_directory = '/content/netflix-prize-data/'

# Function to process and insert data in batches
def process_and_insert_data(filename, collection):
    batch_size = 100000  # Adjust batch size based on your system and data size
    batch = []
    current_movie_id = None

    with open(os.path.join(data_directory, filename), 'r') as file:
        lines = file.readlines()
        current_movie_id = None
        for line in lines:
            if line.endswith(':\n'):  # Movie ID line
                current_movie_id = int(line.strip().replace(':', ''))
            else:
                try:
                    if filename == qualifying_file:
                        customer_id, date = line.strip().split(',')
                        doc = {
                            "movie_id": current_movie_id,
                            "customer_id": int(customer_id),
                            "date": date
                        }
                    elif filename == probe_file:
                        customer_id = int(line.strip())
                        doc = {
                            "movie_id": current_movie_id,
                            "customer_id": customer_id
                        }
                    batch.append(doc)

                    # Insert batch if it reaches batch_size
                    if len(batch) == batch_size:
                        collection.insert_many(batch)
                        batch = []

                except ValueError as e:
                    print(f"Skipping line due to parsing error: {line} - Error: {e}")

        # Insert any remaining documents in the batch
        if batch:
            collection.insert_many(batch)

# Process and insert qualifying data in batches
process_and_insert_data(qualifying_file, qualifying_collection)

# Process and insert probe data in batches
process_and_insert_data(probe_file, probe_collection)


In [ ]:
ratings_collection = db['ratings']
ratings_collection.create_index([("customer_id", pymongo.ASCENDING)])


'customer_id_1'

**Setup Spark environment**

In [ ]:
# scala_map = spark.conf._jconf.getAll()
# python_dict = {}

# iterator = scala_map.iterator()
# while iterator.hasNext():
#     entry = iterator.next()
#     key = entry._1()  # Scala tuple's first element (key)
#     value = entry._2()  # Scala tuple's second element (value)
#     python_dict[key] = value

# print(python_dict)

In [ ]:
!pip install -q findspark


In [ ]:
!pip install pyspark==3.2.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 24.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040918 sha256=88884de89b804803f7c6ca34ae306d8ac3ecd546774845f4d1012f19ec31f891
  Stored in directory: /root/.cache/pip/wheels/e7/e3/c8/c358dac750f2b6a4b03328d10e05a5c69501664bd6504b6c3e
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
!pip install pyngrok


In [ ]:

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, functions as F
from pyspark.sql.functions import *

In [ ]:
!export JAVA_HOME=/usr
!export PATH=$JAVA_HOME/bin:$PATH
!export SPARK_HOME=/usr/local/lib/python3.10/dist-packages/pyspark
!export PATH=$SPARK_HOME/bin:$PATH

In [ ]:
# Configure Spark to use more memory
conf = SparkConf() \
    .setAppName("NetflixRecommendation") \
    .set("spark.driver.memory", "4g") \
    .set("spark.executor.memory", "4g") \
    .set("spark.mongodb.read.connection.uri", "mongodb://192.168.0.112:8081/netflix") \
    .set("spark.mongodb.write.connection.uri", "mongodb://192.168.0.112:8081/netflix") \
    .set("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector:10.0.2") \
    .set("spark.ui.port", "4050")

In [ ]:
# Initialize Spark session
spark = SparkSession.builder.config(conf=conf).getOrCreate()

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2ioatyYmk1ldEQPhNsFZlaRiOgb_7Wjbjommity4XFKATCUgX")
# Open a tunnel to the Spark UI port
public_url = ngrok.connect(4050)
print(f"Spark UI is available at: {public_url}")

Spark UI is available at: NgrokTunnel: "https://a5a9-35-229-232-205.ngrok-free.app" -> "http://localhost:4050"


In [ ]:
ratings_df = spark.read.option("spark.mongodb.database", "netflix") \
.option("spark.mongodb.collection", "ratings") \
.option("partitioner", "com.mongodb.spark.sql.connector.read.partitioner.SamplePartitioner") \
.option("partitioner.options.partition.size","512") \
.load(format='mongodb')

In [ ]:
ratings_df.show()

+--------------------+-----------+----------+--------+------+
|                 _id|customer_id|      date|movie_id|rating|
+--------------------+-----------+----------+--------+------+
|6685992708170e9aa...|    1488844|2005-09-06|       1|     3|
|6685992708170e9aa...|     822109|2005-05-13|       1|     5|
|6685992708170e9aa...|     885013|2005-10-19|       1|     4|
|6685992708170e9aa...|      30878|2005-12-26|       1|     4|
|6685992708170e9aa...|     823519|2004-05-03|       1|     3|
|6685992708170e9aa...|     893988|2005-11-17|       1|     3|
|6685992708170e9aa...|     124105|2004-08-05|       1|     4|
|6685992708170e9aa...|    1248029|2004-04-22|       1|     3|
|6685992708170e9aa...|    1842128|2004-05-09|       1|     4|
|6685992708170e9aa...|    2238063|2005-05-11|       1|     3|
|6685992708170e9aa...|    1503895|2005-05-19|       1|     4|
|6685992708170e9aa...|    2207774|2005-06-06|       1|     5|
|6685992708170e9aa...|    2590061|2004-08-12|       1|     3|
|6685992

In [ ]:
# Preprocess the data if necessary
ratings_df = ratings_df.withColumn("customer_id", ratings_df["customer_id"].cast("integer"))
ratings_df = ratings_df.withColumn("movie_id", ratings_df["movie_id"].cast("integer"))
ratings_df = ratings_df.withColumn("rating", ratings_df["rating"].cast("float"))

In [ ]:
# Initialize ALS model
from pyspark.ml.recommendation import ALS

In [ ]:
als = ALS(
    maxIter=10,
    regParam=0.01,
    userCol="customer_id",
    itemCol="movie_id",
    ratingCol="rating",
    coldStartStrategy="drop",
    nonnegative=True
)

In [ ]:
# Train the ALS model on the entire dataset
model = als.fit(ratings_df)

In [ ]:
# Extract user and item factors
user_factors = model.userFactors
item_factors = model.itemFactors

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
# Save user factors to MongoDB
user_factors.write \
    .format("mongodb") \
    .mode("overwrite") \
    .option("spark.mongodb.database", "netflix") \
    .option("spark.mongodb.collection", "user_factors") \
    .save()

In [ ]:
# Save item factors to MongoDB
item_factors.write \
    .format("mongodb") \
    .mode("overwrite") \
    .option("spark.mongodb.database", "netflix") \
    .option("spark.mongodb.collection", "item_factors") \
    .save()

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Path to save the ALS model in Google Drive
model_path = '/content/drive/MyDrive/nrs1'

# Save the ALS model using Spark's native format
model.save(model_path)


In [ ]:
spark.stop()

**Check if saved model and loaded model are same**

In [ ]:
from pyspark.ml.recommendation import ALSModel

# Path to save the ALS model in Google Drive
model_path = '/content/drive/MyDrive/nrs1'

# Load the ALS model from the saved path in Google Drive
loaded_model = ALSModel.load(model_path)


In [ ]:
# Compare model attributes
if (loaded_model.userCol == model.userCol and
    loaded_model.itemCol == model.itemCol and
    loaded_model.rank == model.rank):
    print("Models have identical parameters.")
else:
    print("Models have different parameters.")

Models have identical parameters.


**Caching Data in Redis**

In [ ]:
!pip install redis pymongo

In [ ]:
import redis
import pymongo
import pickle
import json

In [ ]:
# Connect to MongoDB
mongo_uri = "mongodb://192.168.0.112:8081/"
mongo_client = pymongo.MongoClient(mongo_uri)
db = mongo_client["netflix"]

In [ ]:
# Connect to Redis
redis_url = "192.168.0.112"
redis_port = 8088
redis_client = redis.StrictRedis(host=redis_url, port=redis_port, db=0)

In [ ]:
redis_client.flushdb()

True

In [ ]:
# Function to fetch user factors from MongoDB and cache in Redis
def cache_user_factors():
    user_factors_collection = db['user_factors']
    user_factors = list(user_factors_collection.find({}))  # Retrieve all user factors

    # Use Redis pipeline for bulk update
    pipeline = redis_client.pipeline()

    # Iterate through each user factor and add to pipeline for bulk update
    for user in user_factors:
        user_id = str(user['id'])
        user_factors_data = {
            'features': json.dumps(user['features'])  # Ensure 'features' is serialized to JSON
        }
        pipeline.hset(f'user_factors:{user_id}', mapping=user_factors_data)

    # Execute the pipeline to perform bulk update
    pipeline.execute()

In [ ]:
cache_user_factors()


In [ ]:
# Function to fetch item factors from MongoDB and cache in Redis
def cache_item_factors():
    item_factors_collection = db['item_factors']
    item_factors = list(item_factors_collection.find({}))  # Retrieve all item factors

    # Prepare item factors dictionary
    item_factors_dict = {}
    for item in item_factors:
        item_id = str(item['id'])
        item_factors_dict[item_id] = item['features']

    # Store item factors dictionary as JSON in Redis
    redis_client.set('item_factors', json.dumps(item_factors_dict))

In [ ]:
cache_item_factors()


In [ ]:
# Function to fetch movies from MongoDB and cache in Redis
def cache_movies():
    movies_collection = db['movies']
    movies = list(movies_collection.find({}))  # Retrieve all movies

    # Use Redis pipeline for bulk update
    pipeline = redis_client.pipeline()

    # Prepare movie data dictionary and add to pipeline for bulk update
    for movie in movies:
        movie_id = str(movie['movie_id'])

        # Validate and convert title and year_of_release
        title = movie.get('title', '')
        year_of_release = movie.get('year_of_release', '')

        # Ensure title and year_of_release are not None
        if year_of_release is None:
            year_of_release = "None"

        movie_data = {
            'title': title,
            'year_of_release': year_of_release
        }
        pipeline.hset(f'movie:{movie_id}', mapping=movie_data)

    # Execute the pipeline to perform bulk update
    pipeline.execute()

In [ ]:
cache_movies()


In [ ]:
# Function to cache user IDs in Redis
def cache_user_ids():
    user_factors_collection = db['user_factors']
    user_factors = list(user_factors_collection.find({}))
    # Extract unique user IDs
    user_ids = [str(user['id']) for user in user_factors]

    # Store user IDs in Redis set 'user_ids'
    redis_client.sadd('user_ids', *user_ids)

In [ ]:
cache_user_ids()

In [ ]:
!pip install IMDbPY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
# from imdb import IMDb

# # Initialize IMDb instance
# ia = IMDb()

# # Function to fetch movie poster from IMDb
# def fetch_movie_poster(title):
#     search_results = ia.search_movie(title)
#     if search_results:
#         movie = search_results[0]
#         ia.update(movie, info=['main'])
#         if 'cover url' in movie:
#             return movie['cover url']
#     return None

# # Function to fetch movies from MongoDB and cache in Redis with posters
# def cache_movies_with_poster():
#     movies_col = db['movies']
#     movies_with_poster_col = db['movies_with_poster']

#     # Use Redis pipeline for bulk update
#     pipeline = redis_client.pipeline()

#     # Fetch movies from MongoDB
#     movies = list(movies_col.find({}))

#     # Prepare movie data dictionary and add to pipeline for bulk update
#     for movie in movies:
#         movie_id = str(movie['movie_id'])

#         # Validate and convert title and year_of_release
#         title = movie.get('title', '')
#         year_of_release = movie.get('year_of_release', '')

#         # Fetch movie poster from IMDb
#         poster_url = fetch_movie_poster(title)

#         # Prepare data to store in MongoDB with poster info
#         movie_data = {
#             'movie_id': movie_id,
#             'title': title,
#             'year_of_release': year_of_release if year_of_release else "None",
#             'poster_url': poster_url if poster_url else "None"
#         }

#         # Store in MongoDB collection with posters
#         movies_with_poster_col.update_one({'movie_id': movie_id}, {'$set': movie_data}, upsert=True)

#         # Update Redis cache with movie details
#         pipeline.hset(f'movie:{movie_id}', mapping=movie_data)

#     # Execute the pipeline to perform bulk update in Redis
#     pipeline.execute()

In [ ]:
# cache_movies_with_poster()